# Loads the dataset

In [34]:
import pandas as pd
import numpy as np

output_2Y = pd.read_csv('output_2Y.csv')
print(output_2Y.shape)
output_2Y.head(3)

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
len(output_2Y['ID'].unique())

In [ ]:
output = output_2Y.sort_values(['ID', 'SECUENCIA']).reset_index(drop=True)

In [ ]:
resol_mask = output['RESOLUTOR'] == 'RESOLUTOR-PUESTO-USUARIO'

In [ ]:
output_spu = output.loc[pd.concat([resol_mask, output['ID']], axis=1).groupby('ID').transform('sum')['RESOLUTOR'] > 0].copy()

In [ ]:
output_spu.shape

In [ ]:
len(output_spu['ID'].unique())

In [ ]:
output_spu.dtypes

In [ ]:
np.any(output_spu['PADRE'].unique() == 0)

# Preprocess and encode

In [ ]:
output_spu.loc[:,'PADRE'] = output_spu.loc[:,'PADRE'].fillna(0)

In [ ]:
import EventLog as el
spulog = el.EventLog(output_spu, 'ID', 'FECHA')

In [ ]:
spulog.static_columns()

In [ ]:
spulog.df.describe()

In [ ]:
[np.unique(spulog.df[x]) for x in ['CANCELADA', 'RECLAMACION', 'SOPORTE', 'REAPERTURAINC']]

In [ ]:
spulog.df.select_dtypes('object').describe()

In [ ]:
spulog.df['GRUPOAUTOR'].fillna('0', inplace=True)

In [ ]:
np.sum((spulog.df['MOTIVOCIERRE'].value_counts()/len(spulog.df)*100) > 0.1)

In [ ]:
np.sum(spulog.df.CENTRO.value_counts()/len(spulog.df)*100 > 0.1)

In [ ]:
np.sum(spulog.df.CENTRO.value_counts()/len(spulog.df)*100>0.1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

centros = spulog.df.CENTRO.value_counts()/len(spulog.df) * 100
centros_encoder = OneHotEncoder(sparse=False, categories=[centros[centros > 0.1].index.values], handle_unknown='ignore')

encoder = el.LogEncoder(transformers = [('static_drop', 'drop', ['SISTEMA']),
                                     ('static_keep', 'keep', ['SOPORTE','ID']),
                                     ('static_onehot', el.WrapperEncoder(spulog.id_column,OneHotEncoder(sparse=False)), ['ORIGEN']),
                                     ('static_onehot_filter', el.WrapperEncoder(spulog.id_column,centros_encoder), ['CENTRO']),
                                     ('dynamic_drop', 'drop', ['FECHAMOD', 'DESCORG', 'ASUNTO', 'SECUENCIA', 'PADRE', 'AREA']),
                                     ('dynamic_keep', 'keep', ['CANCELADA', 'RECLAMACION','REAPERTURAINC']),
                                     ('dynamic_freq', el.FrequencyEncoder(spulog.id_column), ['NODO', 'RESOLUTOR', 'TIPOLOGIA', 'PRIORIDAD', 'GRUPOAUTOR', 'ESTADO', 'TYPEORG']),
                                     ('timestamp', el.TimestampFeatures(spulog.id_column, ['event_order', 'time_from_start', 'elapsed_time_from_event','remaining_time']), [spulog.timestamp_column])])


encoder.check_unused(spulog)

In [ ]:
dataset = encoder.fit_transform(spulog)
dataset.head(5)

In [ ]:
np.all(dataset.isnull().sum()==0)

In [ ]:
dataset.describe

In [ ]:
dataset.columns.values

# Split into train, validation, test

In [ ]:
# Split into train, validation, test

from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import safe_indexing
from itertools import chain


def group_train_test_split(X, Y, group, test_size=0.2):
    train, test = next(GroupShuffleSplit(test_size=test_size, random_state=0).split(X, Y, group))
    arrays = [X, Y, group]

    return list(chain.from_iterable((safe_indexing(a, train),
                                     safe_indexing(a, test)) for a in arrays))

In [ ]:
incidents_X = dataset

# Choose the target attribute
incidents_Y = dataset['remaining_time_FECHA']

# And the attribute (id) that is used to split the dataset in train, validation and test
incidents_group = dataset['ID']

incidents_X.shape, incidents_Y.shape, incidents_group.shape

In [ ]:
incidents_X

In [ ]:
# Split into train, validation, test
X_train, X_test, Y_train, Y_test, group_train, group_test = group_train_test_split(incidents_X, incidents_Y, incidents_group, test_size = 0.5)
X_val, X_test, Y_val, Y_test, group_val, group_test = group_train_test_split(X_test, Y_test, group_test, test_size=0.4)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape, X_val.shape, Y_val.shape

In [ ]:
# Quick check that case ids are not in two slots
#np.asarray([item in group_test.unique() for item in group_train.unique()]).any()

# Trains classifier and evaluates them

In [ ]:
Y_train.describe()

In [ ]:
#from sklearn.ensemble import sikit
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_and_evaluate(X_train, X_val, X_test, Y_train, Y_val, Y_test, cols_to_remove = []):
    # Remove some attributes for training
    train_onehot = X_train.drop(cols_to_remove, axis=1)
    val_onehot = X_val.drop(cols_to_remove, axis=1)
    test_onehot = X_test.drop(cols_to_remove, axis=1)

    # Now, we are going to build estimators for the remaining time
    regressor = RandomForestRegressor(random_state=0,n_jobs=-1)
    regressor.fit(train_onehot, Y_train)

    # Predict
    val_predict = regressor.predict(val_onehot)
    test_predict = regressor.predict(test_onehot)
    
    # And evaluate the model
    val_mae = mean_absolute_error(Y_val, val_predict)
    val_mse = mean_squared_error(Y_val, val_predict)
    test_mae = mean_absolute_error(Y_test, test_predict)
    test_mse = mean_squared_error(Y_test, test_predict)

    print("val: ", val_mae, val_mse )
    print("test: ", test_mae, test_mse)
    
    return val_mae, val_mse, test_mae, test_mse

In [ ]:
#columnas a borrar
ctr = ['ID']
print("Start of the experimentation")
train_and_evaluate(X_train, X_val, X_test, Y_train, Y_val, Y_test, cols_to_remove = ctr)

In [ ]:
#All (ALL): all attributes of the original datasets are considered.
#None (NONE): just the basic attributes of the log, such as name of activity and timestamp are considered.
#Automatic (AUTO): attributes detected as relevant for a decision tree algorithm are selected for the experimentation.
#Random (RND): a set of randomly selected attributes are taken into account.
#Context (CTXT): detected context attributes by the method are included.
#Without context (WCTXT): detected context attributes are excluded from the dataset.

# End classifier